In [1]:
import polars as pl
from datetime import datetime
from typing import Tuple, List
import warnings
import sys
import numpy as np
import scipy
import statsmodels.api as sm
from alpha101_prod import CalcAlpha101Factor

POS_RET_PCT_SCALE_THRESHOLD = 0.01

input_path = "data/all_data_1d.parquet"
input_path = "data/all_data_1d_2023.parquet"
output_path = "data/predictions.parquet"
is_prod = False
is_prod = True

In [2]:
def undimensionalize(data, column_name, window = 50):
    data = data.with_columns((pl.col(column_name) / pl.col(column_name).rolling_mean(window) - 1).over('symbol').alias(column_name))
    return data

def standardize_z(data, column_name):
    data = data.with_columns(((pl.col(column_name) - pl.col(column_name).mean()) / pl.col(column_name).std()).over('open_time')
        .alias(column_name))
    return data
def sign(column):
    return pl.when(column > 0).then(1).otherwise(pl.when(column < 0).then(-1).otherwise(0))

In [3]:
def AddPastReturnFactor(input_df: pl.DataFrame, day_num: int) -> pl.DataFrame:
    input_df = input_df.sort(["symbol", "open_time"])
    # 千万注意，这里是计算此时刻相对于前一时刻的return，不能使用未来信息 -> shift(1)是整体往下移动
    for i in range(1, day_num + 1):
        input_df = input_df.with_columns(
            ((pl.col("close") / pl.col("close").shift(i) - 1) * 100)
            .over("symbol")  # Applying the function over each symbol group
            .alias(f"past_{i}day_close_return")
        )

    # 默认使用过去1天的return作为return列
    input_df = input_df.with_columns(pl.col("past_1day_close_return").alias("return"))

    return input_df

In [4]:
def CalcDayPositionScale(
    input_df: pl.DataFrame, day_num: int, trade_long_rank: int, trade_short_rank: int
) -> pl.DataFrame:
    # print(input_df["linear_compound_factor_1day"])
    agg_avg_ret_list = []
    for i in range(1, day_num + 1):
        for side in ["long", "short"]:
            sort_desc = True if side == "long" else False
            trade_rank_num = trade_long_rank if side == "long" else trade_short_rank

            input_df = input_df.with_columns(
                pl.col(f"linear_compound_factor_{i}day")
                .rank(descending=sort_desc)
                .over("open_time")
                .alias(f"symbol_rank_{side}_{i}day")
            ).with_columns(
                pl.when(pl.col(f"symbol_rank_{side}_{i}day") <= trade_rank_num)
                .then(pl.col(f"close_price_fut_{i}day_ret"))
                .otherwise(None)
                .alias(f"total_{side}_value_scale_{i}day")
            )

            cur_agg_avg_ret = (
                input_df.group_by("open_time")
                .agg(
                    pl.col(f"total_{side}_value_scale_{i}day")
                    .mean()
                    .alias(f"fut_mean_{side}_ret_{i}day"),
                )
                .sort("open_time")
            )
            agg_avg_ret_list.append(cur_agg_avg_ret)

    agg_avg_ret_df = agg_avg_ret_list[0]
    for df in agg_avg_ret_list[1:]:
        agg_avg_ret_df = agg_avg_ret_df.join(df, on="open_time", how="right")

    # Sort the final DataFrame by open_time
    agg_avg_ret_df = agg_avg_ret_df.sort("open_time")
    agg_avg_ret_df = agg_avg_ret_df.select(
        pl.col(
            ["open_time"]
            + [col for col in agg_avg_ret_df.columns if col != "open_time"]
        )
    )

    for i in range(1, day_num + 1):
        for side in ["long", "short"]:
            agg_avg_ret_df = agg_avg_ret_df.with_columns(
                pl.col(f"fut_mean_{side}_ret_{i}day")
                .shift(i)
                .alias(f"past_mean_{side}_ret_{i}day"),
            )

    for i in range(1, day_num + 1):
        for side in ["long", "short"]:
            bullish_scale = 1.2 if side == "long" else 0.8
            bearish_scale = 0.8 if side == "long" else 1.2
            agg_avg_ret_df = agg_avg_ret_df.with_columns(
                pl.when(
                    (pl.col(f"past_mean_long_ret_{i}day") > POS_RET_PCT_SCALE_THRESHOLD)
                    & (
                        pl.col(f"past_mean_short_ret_{i}day")
                        > POS_RET_PCT_SCALE_THRESHOLD
                    )
                )
                .then(bullish_scale)
                .when(
                    (pl.col(f"past_mean_long_ret_{i}day") < POS_RET_PCT_SCALE_THRESHOLD)
                    & (
                        pl.col(f"past_mean_short_ret_{i}day")
                        < POS_RET_PCT_SCALE_THRESHOLD
                    )
                )
                .then(bearish_scale)
                .otherwise(1.0)
                .alias(f"{side}_value_scale_{i}day")
            )
    return input_df, agg_avg_ret_df


In [5]:
def AddTotalPosValueScale(
    input_df: pl.DataFrame, day_num: int, trade_long_rank: int, trade_short_rank: int
) -> pl.DataFrame:
    _, day_scale_df = CalcDayPositionScale(
        input_df, day_num, trade_long_rank, trade_short_rank
    )

    # only need the scale columns
    select_col = [col for col in day_scale_df.columns if "scale" in col]
    day_scale_df = day_scale_df.select(pl.col(["open_time"] + select_col))

    # print(f"before join: {input_df}")
    # print(f"day scale df: ", day_scale_df)

    input_df = input_df.join(day_scale_df, on="open_time", how="left")

    # print(f"after join     : {input_df}")
    return input_df, day_scale_df


In [6]:
def AddAmihud(input_df: pl.DataFrame, window_size: int = 10) -> pl.DataFrame:
    # Calculate rolling sums for absolute returns and quote volume
    input_df = input_df.with_columns(
        pl.col("return").abs().rolling_sum(window_size=window_size).over("symbol").alias("rolling_abs_return_sum"),
        pl.col("quote_volume").rolling_sum(window_size=window_size).over("symbol").alias("rolling_quote_volume_sum"),
    )

    # Calculate Amihud illiquidity measure
    input_df = input_df.with_columns(
        (pl.col("rolling_abs_return_sum") / pl.col("rolling_quote_volume_sum"))
        .over("symbol")  # Apply the final operation within each symbol group
        .alias("amihud")
    )

    # Drop intermediate columns
    input_df = input_df.drop(["rolling_abs_return_sum", "rolling_quote_volume_sum"])

    return input_df

In [7]:
def AddReturnAutocorr(input_df: pl.DataFrame, window_size: int = 20, lag: int = 1) -> pl.DataFrame:
    # Calculate rolling sums for absolute returns and quote volume
    input_df = input_df.with_columns(
        pl.rolling_corr(pl.col("return"), pl.col("return").shift(1), window_size=window_size).over(["symbol"]).alias("return_autocorr_" + str(lag))
    )
    return input_df

In [8]:
def AddReturnSkewness(input_df: pl.DataFrame, window_size: int = 20) -> pl.DataFrame:
    # Calculate rolling sums for absolute returns and quote volume
    input_df = input_df.with_columns(
        pl.col("return").rolling_skew(window_size).over(["symbol"]).alias("return_skewness")
    )
    return input_df

In [9]:
def AddTakerBuyRatio(input_df: pl.DataFrame) -> pl.DataFrame:
    input_df = input_df.with_columns(
        (pl.col("taker_buy_volume") / pl.col("volume")).over('symbol').alias("taker_buy_ratio")
    )
    return input_df

In [10]:
def AddID(input_df: pl.DataFrame) -> pl.DataFrame:
    input_df = input_df.with_columns((pl.col('close') / pl.col('open') - 1).over('symbol').alias('ret1'))
    input_df = input_df.sort(by=['symbol', 'open_time'])

    # 计算过去 3天的最大涨幅和最小涨幅
    max_return = pl.col("ret1").rolling_max(window_size=48)
    min_return = pl.col("ret1").rolling_min(window_size=48)
    # 计算 ID
    input_df = input_df.with_columns(
        (sign(pl.col('ret1'))
        * (pl.col("ret1").rolling_max(window_size=48) - pl.col("ret1").rolling_min(window_size=72)) / pl.col('close'))
        .over('symbol')
        .alias("ID")
    )
    input_df = input_df.with_columns(
        pl.col('ID').clip(-0.2, 0.3).alias('ID')
    )
    input_df = input_df.drop('ret1')
    return input_df

In [11]:
def AddAutocorrRank(input_df: pl.DataFrame, window_size: int = 3) -> pl.DataFrame:
    input_df = input_df.with_columns(
        pl.col('close').shift(2).over('symbol').alias('shift_close2')
    )
    input_df = input_df.with_columns([
        pl.col("close").rank().over("symbol").alias("rank_close"),
        pl.col("shift_close2").rank().over("symbol").alias("rank_shift_close2")
    ])
    # 计算排名后的滚动Pearson相关系数，实际上就是滚动的Spearman相关系数
    input_df = input_df.with_columns(
        pl.rolling_corr("rank_close", "rank_shift_close2", window_size=6)
        .over("symbol")
        .alias("spearman_corr2_w3")
    )
    input_df = input_df.drop(["shift_close2","rank_close","rank_shift_close2"])
    return input_df

In [12]:
def beat_ratio(input_df: pl.DataFrame) -> pl.DataFrame:
    input_df = input_df.with_columns([
    (pl.col("quote_volume") - pl.col('taker_buy_quote_volume')).alias("taker_sell_quote_volume")
    ])

    input_df = input_df.with_columns([
        (pl.col("taker_buy_quote_volume") / pl.col("taker_sell_quote_volume")).alias("beat_ratio")
    ])

    return input_df

In [13]:
def factor011(input_df: pl.DataFrame, x1: int, x2: int) -> pl.DataFrame:
    
    input_df = input_df.with_columns([
        (pl.col('quote_volume') / pl.col('volume')).alias('vwap'),
        (pl.col('volume') - (pl.col('volume').shift(x1).over('symbol'))).alias('volume_change')
    ])

    input_df = input_df.with_columns([
        (pl.col('vwap') - pl.col('close')).rolling_max(x2).over('symbol').alias('vwap_close_max'),
        (pl.col('vwap') - pl.col('close')).rolling_min(x2).over('symbol').alias('vwap_close_min')
    ])

    input_df = input_df.with_columns([
        pl.col('vwap_close_max').rank('dense').over('open_time').alias('vwap_close_max_rank'),
        pl.col('vwap_close_min').rank('dense').over('open_time').alias('vwap_close_min_rank'),
        pl.col('volume_change').rank('dense').over('open_time').alias('volume_change_rank')
    ])

    input_df = input_df.with_columns([
        (pl.col('vwap_close_max_rank') + pl.col('vwap_close_min_rank') * pl.col('volume_change_rank')).alias('factor011_rank')
    ])

    return input_df

In [14]:
def nettotal_taker_quote_volume(input_df: pl.DataFrame, N: int) -> pl.DataFrame:

    input_df = input_df.with_columns([
        (pl.col('quote_volume') - pl.col('taker_buy_quote_volume')).alias('taker_sell_quote_volume')
    ])

    input_df = input_df.with_columns([
        pl.col('taker_buy_quote_volume').rolling_sum(N).over('symbol').alias('taker_buy_quote_volume_sum'),
        pl.col('taker_sell_quote_volume').rolling_sum(N).over('symbol').alias('taker_sell_quote_volume_sum'),
    ])

    input_df = input_df.with_columns([
        (pl.col('taker_buy_quote_volume_sum') - pl.col('taker_sell_quote_volume_sum')).alias('net_taker_quote_volume'),
        (pl.col('taker_buy_quote_volume_sum') + pl.col('taker_sell_quote_volume_sum')).alias('total_taker_quote_volume')
    ])

    return input_df

In [15]:
def AddFutureRetCol(input_df: pl.DataFrame, day_num: int) -> pl.DataFrame:
    input_df = input_df.sort(["symbol", "open_time"])
    for i in range(1, day_num + 1):
        input_df = input_df.with_columns(
            ((pl.col("close").shift(-i) / pl.col("close") - 1) * 100)
            .over("symbol")  # Applying the function over each symbol group
            .alias(f"close_price_fut_{i}day_ret")
        )
        input_df = input_df.with_columns(
            ((pl.col("open").shift(-i) / pl.col("open") - 1) * 100)
            .over("symbol")  # Applying the function over each symbol group
            .alias(f"open_price_fut_{i}day_ret")
        )
    return input_df

In [16]:
def ols_get_model(
    data: pl.DataFrame,
    x_col,
    y_col,
    date_threshold: datetime = datetime(2023, 1, 1)
):
    date_threshold_ms = date_threshold.timestamp() * 1000
    train_data = data.filter(
        pl.col("open_time") < date_threshold_ms
    )
    train_data = train_data.drop_nulls()
    x = train_data.select(x_col).cast(pl.Float64).to_numpy().flatten()
    y = train_data.select(y_col).cast(pl.Float64).to_numpy().flatten()
    X = sm.add_constant(x)
    model = sm.OLS(y, X).fit()
    return model

In [17]:
def replace_with_residuals(
    input_df: pl.DataFrame,
    model: sm.OLS,
    x_col: str,
    y_col: str
)-> pl.DataFrame:
    x = input_df.select(x_col).cast(pl.Float64).to_numpy().flatten()
    X = sm.add_constant(x)
    y_hat = model.predict(X)
    y_actual = input_df.select(y_col).cast(pl.Float64).to_numpy().flatten()
    residuals = y_actual - y_hat
    residuals_series = pl.Series('residuals', residuals)
    input_df = input_df.with_columns(residuals_series.alias(y_col))
    
    return input_df

In [18]:
def fama_macbeth_get_factor_weight(
    train_data: pl.DataFrame,
    update_pos_days: int,
    factor_num: int,
    factor_combination_list: List[str],
) -> Tuple[np.ndarray, float]:
    # Drop rows containing any null values
    train_data = train_data.drop_nulls()

    total_weights_sum = np.zeros(factor_num)
    unique_times = train_data.select(pl.col("open_time").sort()).unique().to_numpy()
    print('unique_times', unique_times)
    constant_sum = 0.0

    for each_time in unique_times:
        y_column_name = f"close_price_fut_{update_pos_days}day_ret"
        assert (
            y_column_name in train_data.columns
        ), f"Column {y_column_name} (as y) not found in train data"

        slice_data = train_data.filter(pl.col("open_time") == each_time).fill_nan(0)

        X = slice_data[factor_combination_list].to_numpy()
        X = sm.add_constant(X)  # Add constant term (intercept)
        y = slice_data[y_column_name].to_numpy()

        model = sm.OLS(y, X)
        results = model.fit()
        weights = results.params[1:]
        # print(results.params[1:])
        constant_sum += results.params[0]  # constant term

        while weights.shape[0] < total_weights_sum.shape[0]:
            weights = np.append(weights, 0)

        total_weights_sum += weights
        print(total_weights_sum)
    print('unique_times', len(unique_times))
    total_weights_sum /= len(unique_times)
    avg_const_term = constant_sum / len(unique_times)

    return total_weights_sum, avg_const_term


In [19]:
def CalcLinearCompoundFactor(
    input_df: pl.DataFrame,
    day_num: int,
    factor_combination_list: list,
    date_threshold: datetime = datetime(2025, 4, 1),
    prev_threshold: datetime = datetime(2024, 4, 1),
) -> pl.DataFrame:

    factor_num = len(factor_combination_list)

    for cur_update_position_time in range(1, day_num + 1):
        cur_fut_ret_column_name = f"close_price_fut_{cur_update_position_time}day_ret"

        non_nan_result = input_df.filter(
            (pl.col(cur_fut_ret_column_name).is_not_nan())
            & (pl.col(cur_fut_ret_column_name).is_not_null())
        ).sort(["open_time", "symbol"])

        non_nan_linear_x = non_nan_result.select(
            ["open_time", "symbol", cur_fut_ret_column_name] + factor_combination_list
        )

        date_threshold_ms = date_threshold.timestamp() * 1000
        prev_threshold_ms = prev_threshold.timestamp() * 1000
        
        # 检查每个因子的null值情况
        for factor in factor_combination_list:
            nulls = non_nan_linear_x[factor].null_count()
            print(f"{factor} nulls: {nulls}")
            nans = non_nan_linear_x[factor].is_nan().sum()
            print(f"{factor} nans: {nans}")
        ######    
            
        linear_x_train = non_nan_linear_x.filter(
            pl.col("open_time") < date_threshold_ms
        )
        linear_x_train = linear_x_train.filter(
            pl.col("open_time") > prev_threshold_ms
        )
        print('linear_x_train',linear_x_train.sort("open_time"))
        # # 检查过确实只使用训练数据拟合，没有用测试信息
        # print(linear_x_train.sort(by="open_time"))

        weighted_factors, const_term = fama_macbeth_get_factor_weight(
            linear_x_train,
            cur_update_position_time,
            factor_num=factor_num,
            factor_combination_list=factor_combination_list,
        )
        ##
        # 3. 检查权重计算结果
        def check_weights(weighted_factors, const_term, factor_combination_list):
            print("\nWeight check:")
            print("Constant term:", const_term)
            for factor, weight in zip(factor_combination_list, weighted_factors):
                print(f"{factor}: {weight}")
            
            # 检查权重是否有异常值
            if any(np.isnan(weighted_factors)) or np.isnan(const_term):
                print("Warning: NaN values in weights!")
            if any(np.abs(weighted_factors) > 100):
                print("Warning: Unusually large weights!")
        check_weights(weighted_factors, const_term, factor_combination_list)
        
        weighted_sum_expr = pl.lit(const_term)
        for factor, weight in zip(factor_combination_list, weighted_factors):
            weighted_sum_expr += pl.col(factor) * weight

        # 这个代码用于实际交易的时候，我们只需要使用权重计算未来收益率
        input_df = input_df.with_columns(
            weighted_sum_expr.alias(
                f"linear_compound_factor_{cur_update_position_time}day"
            )
        )
        
        # 4. 检查最终的因子计算结果
        def check_final_factor(input_df, cur_update_position_time):
            factor_col = f"linear_compound_factor_{cur_update_position_time}day"
            print("\nFinal factor check:")
            print(input_df.select([
                pl.col(factor_col).mean().alias('mean'),
                pl.col(factor_col).std().alias('std'),
                pl.col(factor_col).min().alias('min'),
                pl.col(factor_col).max().alias('max'),
                pl.col(factor_col).null_count().alias('nulls')
            ]))
        check_final_factor(input_df, cur_update_position_time)

    return input_df.filter(
        pl.col("open_time") >= date_threshold_ms
    )  # only return the data after the threshold


In [20]:
def AddVolatilityCol(input_df: pl.DataFrame) -> pl.DataFrame:
    def calculate_volatility(group, bar_name: str, window_size: int = 30):
        return group.with_columns(
            [
                pl.col(bar_name)
                .pct_change()
                .rolling_std(window_size=window_size)
                .alias(f"{bar_name}_price_volatility")
            ]
        )

    input_df = input_df.group_by("symbol").map_groups(
        lambda x: calculate_volatility(x, "open", window_size=30)
    )
    input_df = input_df.group_by("symbol").map_groups(
        lambda x: calculate_volatility(x, "close", window_size=30)
    )
    return input_df

In [21]:
FACTOR_COMBINATION_LIST = [
    "amihud",
    "return_skewness",
    "alpha30",
    "alpha36",
    "alpha40",
    "alpha45",
    "ID",
]
UPDATE_POSITION_TIME = 10

input_data = pl.read_parquet(input_path)
input_data = input_data.with_columns(pl.from_epoch(pl.col("open_time"), time_unit="ms").cast(pl.Datetime('ms')).alias("open_time"))
input_data = input_data.with_columns(pl.from_epoch(pl.col("close_time"), time_unit="ms").cast(pl.Datetime('ms')).alias("close_time"))
input_data = input_data.sort(by=["symbol", "open_time"])

# for production need
input_data = input_data.filter(
    ~pl.col("symbol").is_in(
        ["BTCUSDT", "ETHUSDT", "BCHUSDT", "LTCUSDT", "ETCUSDT", "LINKUSDT", "AVAXUSDT", "SOLUSDT"]
    )
).filter(pl.col("symbol").str.ends_with("USDT"))

input_data = AddPastReturnFactor(input_data, day_num=10)
input_data = AddVolatilityCol(input_data)
input_data = AddAmihud(input_data)
# input_data = AddReturnAutocorr(input_data, 28, 1)
input_data = AddReturnSkewness(input_data, 7)

# 填充空值
input_data = input_data.with_columns([
    pl.col("return_skewness").fill_nan(None).over('symbol').alias('return_skewness')
])
# input_data = AddTakerBuyRatio(input_data)  # 1.666
input_data = AddID(input_data)  # 1.895
# 填充空值
input_data = input_data.with_columns([
    pl.col("ID").fill_nan(None).over('symbol').alias('ID')
])
# input_data = AddAutocorrRank(input_data)  # 1.371
# input_data = beat_ratio(input_data)  # 1.540
# input_data = factor011(input_data, 60, 65)  # 1.980
# input_data = nettotal_taker_quote_volume(input_data, 20)  # 1.634

# Calculate alpha101 factor
alpha101_factor_list = [x for x in FACTOR_COMBINATION_LIST if "alpha" in x]
ret_skewness = input_data.sort(['open_time'])['return_skewness']
input_data = CalcAlpha101Factor(input_data, calc_factor_list=alpha101_factor_list)
input_data.sort(['open_time'])

get alpah func alpha30
get alpah func alpha36
get alpah func alpha40
get alpah func alpha45
alpha30
alpha36
alpha40
alpha45


open_time,close_time,open,high,low,close,volume,quote_volume,count,taker_buy_volume,taker_buy_quote_volume,symbol,past_1day_close_return,past_2day_close_return,past_3day_close_return,past_4day_close_return,past_5day_close_return,past_6day_close_return,past_7day_close_return,past_8day_close_return,past_9day_close_return,past_10day_close_return,return,open_price_volatility,close_price_volatility,amihud,return_skewness,ID,alpha30,alpha36,alpha40,alpha45
datetime[ms],datetime[ms],f64,f64,f64,f64,f64,f64,i64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2021-01-01 00:00:00,2021-01-01 23:59:59.999,1.3658,1.4033,1.082,1.1578,4.4448765e7,5.3531e7,302387,2.0164935e7,2.4297e7,"""1INCHUSDT""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.0,0.0,0.0,0.0
2021-01-01 00:00:00,2021-01-01 23:59:59.999,88.581,91.0,84.652,90.909,368132.6,3.2361e7,95220,160420.1,1.4119e7,"""AAVEUSDT""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.0,0.0,0.0,0.0
2021-01-01 00:00:00,2021-01-01 23:59:59.999,0.18158,0.1851,0.16999,0.17517,6.54388233e8,1.1724e8,316213,2.85584091e8,5.1267e7,"""ADAUSDT""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.0,0.0,0.0,0.0
2021-01-01 00:00:00,2021-01-01 23:59:59.999,0.3363,0.4334,0.3315,0.3987,2.24658043e8,8.8468e7,340656,1.0992e8,4.3267e7,"""ALGOUSDT""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.0,0.0,0.0,0.0
2021-01-01 00:00:00,2021-01-01 23:59:59.999,0.18514,0.22069,0.18178,0.21723,3.7146341e7,7.5176e6,46654,1.9126319e7,3.8687e6,"""ALPHAUSDT""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.0,0.0,0.0,0.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2025-05-04 00:00:00,2025-05-04 23:59:59.999,0.2591,0.2594,0.259,0.2594,8054.0,2088.428,25,3967.0,1028.9431,"""ZETAUSDT""",-8.950509,-5.432009,-4.946867,-0.153965,-1.853954,3.429027,-3.353204,-2.810041,-4.527052,-2.627628,-8.950509,0.057279,0.057165,0.000075,-0.44833,0.3,0.000019,2.497629,-0.077698,-0.451174
2025-05-04 00:00:00,2025-05-04 23:59:59.999,0.01237,0.01238,0.01235,0.01235,549142.0,6790.64054,51,106910.0,1322.9158,"""ZILUSDT""",-5.58104,-5.868902,-3.137255,-2.755906,-5.65317,-3.364632,-6.51022,-6.012177,-4.559505,-5.291411,-5.58104,0.038688,0.039016,0.000092,-0.162079,-0.2,0.000417,2.22064,-0.059583,-0.055008
2025-05-04 00:00:00,2025-05-04 23:59:59.999,0.05048,0.0505,0.05046,0.0505,76439.0,3859.43285,61,46346.0,2340.10069,"""ZKUSDT""",-11.898116,-13.925345,-12.417621,-14.537147,-14.911542,-12.84087,-18.271565,-16.071132,-12.280702,-12.036231,-11.898116,0.056253,0.055896,0.000134,-0.83866,0.3,0.000057,3.070752,-0.030071,0.08647


In [22]:
# 截面归一化
input_data = input_data.sort(by=["symbol", "open_time"])
original_columns = input_data.columns
print('original_columns',original_columns)
for c in FACTOR_COMBINATION_LIST:
    print(c)
    input_data = input_data.with_columns(pl.col(c).mean().over("open_time").alias("mean_" + c))
    input_data = input_data.with_columns(pl.col(c).std().over("open_time").alias("std_" + c))
    input_data = input_data.with_columns(((pl.col(c) - pl.col("mean_" + c)) / pl.col("std_" + c)).alias(c))
input_data = input_data.select(original_columns)
input_data.sort(['open_time'])


original_columns ['open_time', 'close_time', 'open', 'high', 'low', 'close', 'volume', 'quote_volume', 'count', 'taker_buy_volume', 'taker_buy_quote_volume', 'symbol', 'past_1day_close_return', 'past_2day_close_return', 'past_3day_close_return', 'past_4day_close_return', 'past_5day_close_return', 'past_6day_close_return', 'past_7day_close_return', 'past_8day_close_return', 'past_9day_close_return', 'past_10day_close_return', 'return', 'open_price_volatility', 'close_price_volatility', 'amihud', 'return_skewness', 'ID', 'alpha30', 'alpha36', 'alpha40', 'alpha45']
amihud
return_skewness
alpha30
alpha36
alpha40
alpha45
ID


open_time,close_time,open,high,low,close,volume,quote_volume,count,taker_buy_volume,taker_buy_quote_volume,symbol,past_1day_close_return,past_2day_close_return,past_3day_close_return,past_4day_close_return,past_5day_close_return,past_6day_close_return,past_7day_close_return,past_8day_close_return,past_9day_close_return,past_10day_close_return,return,open_price_volatility,close_price_volatility,amihud,return_skewness,ID,alpha30,alpha36,alpha40,alpha45
datetime[ms],datetime[ms],f64,f64,f64,f64,f64,f64,i64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2021-01-01 00:00:00,2021-01-01 23:59:59.999,1.3658,1.4033,1.082,1.1578,4.4448765e7,5.3531e7,302387,2.0164935e7,2.4297e7,"""1INCHUSDT""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,NaN,NaN,NaN,NaN
2021-01-01 00:00:00,2021-01-01 23:59:59.999,88.581,91.0,84.652,90.909,368132.6,3.2361e7,95220,160420.1,1.4119e7,"""AAVEUSDT""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,NaN,NaN,NaN,NaN
2021-01-01 00:00:00,2021-01-01 23:59:59.999,0.18158,0.1851,0.16999,0.17517,6.54388233e8,1.1724e8,316213,2.85584091e8,5.1267e7,"""ADAUSDT""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,NaN,NaN,NaN,NaN
2021-01-01 00:00:00,2021-01-01 23:59:59.999,0.3363,0.4334,0.3315,0.3987,2.24658043e8,8.8468e7,340656,1.0992e8,4.3267e7,"""ALGOUSDT""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,NaN,NaN,NaN,NaN
2021-01-01 00:00:00,2021-01-01 23:59:59.999,0.18514,0.22069,0.18178,0.21723,3.7146341e7,7.5176e6,46654,1.9126319e7,3.8687e6,"""ALPHAUSDT""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,NaN,NaN,NaN,NaN
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2025-05-04 00:00:00,2025-05-04 23:59:59.999,0.2591,0.2594,0.259,0.2594,8054.0,2088.428,25,3967.0,1028.9431,"""ZETAUSDT""",-8.950509,-5.432009,-4.946867,-0.153965,-1.853954,3.429027,-3.353204,-2.810041,-4.527052,-2.627628,-8.950509,0.057279,0.057165,-0.181059,-0.855366,2.322289,-0.068531,0.258641,-0.087425,-1.388506
2025-05-04 00:00:00,2025-05-04 23:59:59.999,0.01237,0.01238,0.01235,0.01235,549142.0,6790.64054,51,106910.0,1322.9158,"""ZILUSDT""",-5.58104,-5.868902,-3.137255,-2.755906,-5.65317,-3.364632,-6.51022,-6.012177,-4.559505,-5.291411,-5.58104,0.038688,0.039016,-0.144981,-0.387953,-0.838496,-0.056903,0.036302,0.008318,-0.17399
2025-05-04 00:00:00,2025-05-04 23:59:59.999,0.05048,0.0505,0.05046,0.0505,76439.0,3859.43285,61,46346.0,2340.10069,"""ZKUSDT""",-11.898116,-13.925345,-12.417621,-14.537147,-14.911542,-12.84087,-18.271565,-16.071132,-12.280702,-12.036231,-11.898116,0.056253,0.055896,-0.054544,-1.492728,2.322289,-0.067407,0.71869,0.164297,0.259736


In [23]:
# below for combine factors
print(f"begin to calc linear compound factor: {FACTOR_COMBINATION_LIST}")
input_data = AddFutureRetCol(input_data, UPDATE_POSITION_TIME)
input_data

input_data = CalcLinearCompoundFactor(
    input_data, UPDATE_POSITION_TIME, FACTOR_COMBINATION_LIST
)

input_data, day_scale_df = AddTotalPosValueScale(
    input_data, day_num=10, trade_long_rank=20, trade_short_rank=10
)

if is_prod:
    # for normal run, save all data
    print('normal run')
    results = input_data
else:
    # for backtest and research, remove the last few rows and symbols whose min value is larger then 5 USDT
    print('backtest and research run')
    results = input_data.filter(pl.col("close_price_fut_7day_ret").is_not_null())
results.write_parquet(output_path)
results

begin to calc linear compound factor: ['amihud', 'return_skewness', 'alpha30', 'alpha36', 'alpha40', 'alpha45', 'ID']
amihud nulls: 3250
amihud nans: 0
return_skewness nulls: 14693
return_skewness nans: 0
alpha30 nulls: 0
alpha30 nans: 1330
alpha36 nulls: 0
alpha36 nans: 17546
alpha40 nulls: 0
alpha40 nans: 630
alpha45 nulls: 0
alpha45 nans: 1684
ID nulls: 22990
ID nans: 0
linear_x_train shape: (107_043, 10)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ open_time ┆ symbol    ┆ close_pri ┆ amihud    ┆ … ┆ alpha36   ┆ alpha40   ┆ alpha45   ┆ ID       │
│ ---       ┆ ---       ┆ ce_fut_1d ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---      │
│ datetime[ ┆ str       ┆ ay_ret    ┆ f64       ┆   ┆ f64       ┆ f64       ┆ f64       ┆ f64      │
│ ms]       ┆           ┆ ---       ┆           ┆   ┆           ┆           ┆           ┆          │
│           ┆           ┆ f64       ┆           ┆   ┆           ┆           ┆      

open_time,close_time,open,high,low,close,volume,quote_volume,count,taker_buy_volume,taker_buy_quote_volume,symbol,past_1day_close_return,past_2day_close_return,past_3day_close_return,past_4day_close_return,past_5day_close_return,past_6day_close_return,past_7day_close_return,past_8day_close_return,past_9day_close_return,past_10day_close_return,return,open_price_volatility,close_price_volatility,amihud,return_skewness,ID,alpha30,alpha36,alpha40,alpha45,close_price_fut_1day_ret,open_price_fut_1day_ret,close_price_fut_2day_ret,open_price_fut_2day_ret,close_price_fut_3day_ret,…,open_price_fut_7day_ret,close_price_fut_8day_ret,open_price_fut_8day_ret,close_price_fut_9day_ret,open_price_fut_9day_ret,close_price_fut_10day_ret,open_price_fut_10day_ret,linear_compound_factor_1day,linear_compound_factor_2day,linear_compound_factor_3day,linear_compound_factor_4day,linear_compound_factor_5day,linear_compound_factor_6day,linear_compound_factor_7day,linear_compound_factor_8day,linear_compound_factor_9day,linear_compound_factor_10day,long_value_scale_1day,short_value_scale_1day,long_value_scale_2day,short_value_scale_2day,long_value_scale_3day,short_value_scale_3day,long_value_scale_4day,short_value_scale_4day,long_value_scale_5day,short_value_scale_5day,long_value_scale_6day,short_value_scale_6day,long_value_scale_7day,short_value_scale_7day,long_value_scale_8day,short_value_scale_8day,long_value_scale_9day,short_value_scale_9day,long_value_scale_10day,short_value_scale_10day
datetime[ms],datetime[ms],f64,f64,f64,f64,f64,f64,i64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2025-04-01 00:00:00,2025-04-01 23:59:59.999,0.011252,0.012753,0.011172,0.012493,7.9170e9,9.5172e7,595014,3.9677e9,4.7755e7,"""1000BONKUSDT""",11.009419,11.654303,10.274517,2.033649,-8.024737,-10.251437,-10.904293,-11.416011,-9.98631,3.213814,11.009419,0.072392,0.069732,-0.302524,1.794147,0.946675,-0.428637,0.344232,-0.13713,-0.318906,-11.75058,11.02915,-12.599055,-1.990757,-9.013047,…,-9.607181,-11.198271,-15.35727,-14.968382,-1.404195,0.216121,-5.599005,-0.231799,-0.470033,-0.691868,-0.86583,-1.055189,-1.210641,-1.396314,-1.565765,-1.779879,-1.978124,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2025-04-02 00:00:00,2025-04-02 23:59:59.999,0.012493,0.013,0.010796,0.011025,1.4604e10,1.7727e8,1062129,7.1800e9,8.7204e7,"""1000BONKUSDT""",-11.75058,-2.034832,-1.465725,-2.683379,-9.955897,-18.832364,-20.797414,-21.373556,-21.825144,-20.563441,-11.75058,0.069734,0.062942,-0.368663,1.762249,-0.58263,-0.924966,0.016325,-0.145396,-0.691544,-0.961451,-11.726567,3.102041,-12.599055,0.770975,…,-23.765309,-3.646259,-11.198271,13.560091,-14.976387,25.913832,0.224126,-0.261941,-0.494046,-0.708583,-0.955353,-1.184319,-1.349946,-1.527509,-1.776865,-1.959361,-2.183228,0.8,1.2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2025-04-03 00:00:00,2025-04-03 23:59:59.999,0.011028,0.01144,0.010395,0.010919,4.7304e9,5.1591e7,426744,2.3793e9,2.5955e7,"""1000BONKUSDT""",-0.961451,-12.599055,-2.976719,-2.413084,-3.619031,-10.821627,-19.612751,-21.558908,-22.129511,-22.576757,-0.961451,0.062958,0.062547,-0.338566,1.473211,-1.205945,0.572749,0.391702,-0.230299,-0.314003,4.10294,-0.988393,1.749244,3.073993,-10.147449,…,0.598477,14.662515,-3.681538,27.136185,13.538266,15.102116,25.888647,-0.243802,-0.489366,-0.705201,-0.96995,-1.176003,-1.371593,-1.591554,-1.901376,-2.109024,-2.359953,1.0,1.0,0.8,1.2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2025-04-04 00:00:00,2025-04-04 23:59:59.999,0.010919,0.011503,0.010677,0.011367,5.0756e9,5.6471e7,456907,2.4482e9,2.7239e7,"""1000BONKUSDT""",4.10294,3.102041,-9.013047,1.004087,1.590848,0.335422,-7.162692,-16.314511,-18.340517,-18.934531,4.10294,0.062564,0.062753,-0.335486,1

In [24]:
results
unique_symbols = input_data.select(pl.col("symbol").unique())
symbol_list = unique_symbols["symbol"].to_list()
print(symbol_list)

['AIUSDT', 'XEMUSDT', 'ETHFIUSDT', 'ONEUSDT', 'APTUSDT', 'AGIXUSDT', 'IMXUSDT', 'COSUSDT', 'FTMUSDT', 'REIUSDT', 'CVXUSDT', 'CVCUSDT', 'VANRYUSDT', 'ORDIUSDT', 'DOGSUSDT', 'WAVESUSDT', 'LOKAUSDT', 'RONINUSDT', 'SXPUSDT', 'FIOUSDT', 'POLYXUSDT', 'PERPUSDT', 'IOUSDT', 'UMAUSDT', 'AUCTIONUSDT', 'SANTOSUSDT', 'ATOMUSDT', 'NEIROETHUSDT', 'NMRUSDT', 'SCRUSDT', 'JTOUSDT', 'TURBOUSDT', 'ZKUSDT', 'BNXUSDT', 'BNTUSDT', 'XTZUSDT', 'BSWUSDT', 'AEVOUSDT', 'ORBSUSDT', 'WIFUSDT', 'LUNA2USDT', 'MINAUSDT', 'KAVAUSDT', 'JASMYUSDT', 'KNCUSDT', 'RUNEUSDT', 'DARUSDT', 'ALPACAUSDT', 'GRTUSDT', 'PENDLEUSDT', 'ARKUSDT', 'VOXELUSDT', 'IOTAUSDT', 'EDUUSDT', 'BONDUSDT', 'TNSRUSDT', 'JOEUSDT', 'HOOKUSDT', 'ACEUSDT', 'MANTAUSDT', 'YFIUSDT', 'CELRUSDT', 'RLCUSDT', 'GLMRUSDT', 'ICXUSDT', 'ONDOUSDT', 'POWRUSDT', 'ZRXUSDT', 'FIDAUSDT', 'OMUSDT', 'CHRUSDT', 'NTRNUSDT', 'MEWUSDT', 'LEVERUSDT', 'BICOUSDT', 'APEUSDT', 'OXTUSDT', '1INCHUSDT', 'ARBUSDT', 'CYBERUSDT', 'FLUXUSDT', 'LSKUSDT', 'ETHWUSDT', 'ARPAUSDT', 'HOTUSDT',

In [25]:
ret_skewness.to_pandas().describe()

count    277756.000000
mean          0.028340
std           0.691188
min          -2.041241
25%          -0.435900
50%           0.015277
75%           0.484170
max           2.041241
Name: return_skewness, dtype: float64